<a href="https://colab.research.google.com/github/KAATHEEK/3D-Segmentation-Task/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Replace 'your_zip_file.zip' with the actual name of your zip file.
zip_file_path = '/content/drive/MyDrive/FLARE22Train.zip'

# Specify the target directory where you want to extract the contents.
# You can change the 'target_directory' as needed.
target_directory = 'your_target_directory1'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

# List the contents of the target directory to verify the extraction.
import os
extracted_files = os.listdir(target_directory)
print(extracted_files)

['FLARE22Train']


In [ ]:
pip install numpy nibabel scikit-image tensorflow keras

In [ ]:
pip install torchio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 9.4 MB/s eta 0:00:00


In [ ]:
pip install interpolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
from torchvision.transforms import functional as TF

# Assuming a simple VNet architecture
class VNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VNet, self).__init__()
        # Define the layers of the VNet architecture here
        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=3, padding=1) # Use Conv3d for 3D data
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv3d(64, out_channels, kernel_size=3, padding=1) # Use Conv3d for 3D data

    def forward(self, x):
        # Define forward pass here
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return x

# Define your Dataset
class CTScanDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.image_files = sorted(os.listdir(images_dir))
        self.label_files = sorted(os.listdir(labels_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.image_files[idx])
        label_path = os.path.join(self.labels_dir, self.label_files[idx])

        image = nib.load(img_path).get_fdata()
        label = nib.load(label_path).get_fdata()

        if self.transform:
            image, label = self.transform(image, label)

        return image, label

# Example Transform Function
def transform(image, label):
    # Pad the image and label to ensure consistent depth
    pad_value = 0  # Value to use for padding
    max_depth = 128 # Example desired depth

    current_depth = image.shape[-1]
    pad_required = max_depth - current_depth

    if pad_required > 0:
      pad_width = ((0, 0), (0, 0), (pad_required // 2, pad_required - pad_required // 2))
      image = np.pad(image, pad_width=pad_width, mode='constant', constant_values=pad_value)
      label = np.pad(label, pad_width=pad_width, mode='constant', constant_values=pad_value)

    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    label = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
    return image, label

In [ ]:
images_dir = '/content/your_target_directory1/FLARE22Train/images'
labels_dir = '/content/your_target_directory1/FLARE22Train/labels'

train_dataset = CTScanDataset(images_dir, labels_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [ ]:
# Initialize model, loss function, optimizer
model = VNet(in_channels=1, out_channels=4)  # Assuming 1 input channel and 4 output channels for the organs
criterion = nn.CrossEntropyLoss()  # Or Dice loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
def validate_model(model, val_loader):
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            # Compute Dice score here

validate_model(model, train_loader)  # Example validation call


In [ ]:
torch.save(model.state_dict(), 'vnet_model.pth')
# Add code to visualize the 3D segmented organs
